In [1]:
import warnings
import astropy.units as u
from zoneinfo import ZoneInfo
from astropy.time import Time
from astroplan import Observer
from datetime import datetime, timezone
from astropy.coordinates import EarthLocation, Angle

warnings.filterwarnings('ignore')

In [2]:
observation_night = '2023-01-01'

In [3]:
T80S_TZ = ZoneInfo('America/Santiago')
UTC_TZ = timezone.utc
T80S_LAT = '-30.1678638889 degrees'
T80S_LON = '-70.8056888889 degrees'
T80S_HEI = 2187
t80s_lat = Angle(T80S_LAT, 'deg')
t80s_lon = Angle(T80S_LON, 'deg')
t80s_hei = T80S_HEI*u.m
t80s_EL = EarthLocation(lat=t80s_lat, lon=t80s_lon, height=t80s_hei)

In [4]:
# DATETIME
dt_obs = datetime.fromisoformat(observation_night +' 23:59:59.000+00:00')
t80s_dt = dt_obs.astimezone(T80S_TZ)
t80s_midnight = t80s_dt - t80s_dt.utcoffset()
t80s_Time = Time(t80s_midnight, location=t80s_EL)

In [5]:
# OBSERVER
t80s_obs = Observer(location=t80s_EL, timezone=T80S_TZ)
t80s_Time_mn = t80s_obs.midnight(t80s_Time)

In [6]:
moon_illum = t80s_obs.moon_illumination(t80s_Time_mn)
twilight_morning = t80s_obs.twilight_morning_astronomical(t80s_Time, which='nearest').datetime 
twilight_evening = t80s_obs.twilight_evening_astronomical(t80s_Time, which='nearest').datetime 
night_duration = (twilight_morning - twilight_evening).total_seconds()/3600
lst_even = t80s_obs.local_sidereal_time(twilight_evening)
lst_morn = t80s_obs.local_sidereal_time(twilight_morning)

In [7]:
# Final DIGEST
print(f'Evening twilight:')
print(f'\tDATETIME (UTC):', twilight_evening)
print(f'\tLST:', lst_even)
print(f'Morning twilight:')
print(f'\tDATETIME (UTC):', twilight_morning)
print(f'\tLST:', lst_morn)
print(f'Night duration: {night_duration:.2f} hours')
print(f'Moon illumination: {moon_illum:.4f}')
print(f'Night duration: {night_duration:.2f} hours')

Evening twilight:
	DATETIME (UTC): 2023-01-02 01:24:23.207661
	LST: 3h26m53.51370626s
Morning twilight:
	DATETIME (UTC): 2023-01-02 08:09:41.051338
	LST: 10h13m17.93840116s
Night duration: 6.75 hours
Moon illumination: 0.8032
Moon illumination: 80.32%
Night duration: 6.75 hours
